In [2]:
import PyQt6.QtCore
import os
os.environ["QT_API"] = "pyqt5"


# Use qt backend for matplotlab to use interactive mne plots
%matplotlib qt

In [3]:
import mne 
import analysis.processing
import matplotlib.pyplot as plt
import pandas as pd
import csv 

mne.set_log_level(verbose='WARNING', return_old_level=False, add_frames=None)

In [4]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 200 # 200 

In [36]:
# read group evokeds

features = []
labels = []
groups = []

for pnum in (151, 152, 153, 161, 162, 163, 171, 172, 173, 181,\
                182, 183, 191, 192, 193, 201, 202, 203, 211, 212, 213,\
                    221, 222, 223, 231, 232, 233, ):
    raw = analysis.processing.loadData(pnum)

    # preprocessig 
    raw = analysis.processing.preprocessing(raw)
    epochs, event_dict = analysis.processing.eventEpocshByBlocks(raw)

    oddball_epochs = epochs["oddball or control"]
    stims = oddball_epochs.events[:,-1]
    data =  oddball_epochs.get_data()

    stims[stims == 7] = 1 # odd
    stims[stims == 8 ] = 0 # standard

    features.append(data)
    labels.append(stims)
    groups.append([abs(pnum//10)]*(len(stims)))

/Users/erangad/Documents/Research/oddball-task/src/analysis/processing.py:46: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_filtered.set_montage(montage)
/var/folders/x4/6xjb_tfs6vdg6llm0_l4vjz00000gn/T/ipykernel_7413/3758078117.py:18: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data =  oddball_epochs.get_data()
/Users/erangad/Documents/Research/oddball-task/src/analysis/processing.py:46: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_filtered.set_montage(montage)
/var/folders/x4/6xjb_tfs6vdg6llm0_l4vjz00000gn/T/ipykernel_7413/3758078117.py:18: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data =  oddball_epochs.get_data()
/Users/erangad/Documents/Research/oddball-task/src/analysis/proces

In [37]:
import numpy as np

data_array=np.vstack(features)
label_array=np.hstack(labels)
group_array=np.hstack(groups)
print(data_array.shape,label_array.shape,group_array.shape)

(3240, 62, 201) (3240,) (3240,)


In [38]:
import pickle

# with open('features.pickle', 'wb') as output:
#     pickle.dump(data_array, output)


# with open('labels.pickle', 'wb') as output:
#     pickle.dump(label_array, output)


# with open('groups.pickle', 'wb') as output:
#     pickle.dump(group_array, output)   

In [88]:
unique, counts = np.unique(group_array, return_counts=True)
unique, counts

(array([-27, -26, -25, -24, -23, -22, -21, -20, -19]),
 array([360, 360, 360, 360, 360, 360, 360, 360, 360]))

In [97]:
#import os.path
if (os.path.isfile('features.pickle') and os.path.isfile('labels.pickle') \
    and os.path.isfile('groups.pickle')):
     
     with open('features.pickle', 'rb') as pickle_file:
        data_array = pickle.load(pickle_file)

     with open('labels.pickle', 'rb') as pickle_file:
        label_array = pickle.load(pickle_file)

     with open('groups.pickle', 'rb') as pickle_file:
        group_array = pickle.load(pickle_file)

# replace Pant with Ishan
group_array = np.asarray([i - 14 for i in group_array])

In [98]:
np.unique(group_array)

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [99]:
print(data_array.shape,label_array.shape,group_array.shape)

(3240, 62, 201) (3240,) (3240,)


In [52]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold,GridSearchCV,cross_val_score,cross_validate 
from sklearn.model_selection import LeaveOneGroupOut

In [59]:
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.preprocessing import StandardScaler

class Scaler(BaseEstimator,TransformerMixin):

    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self,X,y=None):
        self.scaler.fit(X.reshape(X.shape[0], -1))
        return self

    def transform(self,X):
        return self.scaler.transform(X.reshape(X.shape[0], -1)).reshape(X.shape)

In [62]:
data_array[1].shape

(62, 201)

In [101]:
clf=svm.SVC()
logo = LeaveOneGroupOut()
param_grid = {'classifier__C': [0.01,0.05,0.1,0.5, 1,2,3,4,5,8, 10,12,15], \
              'classifier__kernel': ('rbf', 'linear')}
pipe=Pipeline([('scaler',Scaler()), ('classifier',clf)])
gscv=GridSearchCV(pipe,param_grid,cv=logo,n_jobs=16)
gscv.fit(data_array,label_array,groups=group_array)

ValueError: 
All the 234 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
234 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/erangad/Documents/Research/oddball-task/.venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/erangad/Documents/Research/oddball-task/.venv/lib/python3.9/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/erangad/Documents/Research/oddball-task/.venv/lib/python3.9/site-packages/sklearn/pipeline.py", line 427, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/erangad/Documents/Research/oddball-task/.venv/lib/pyth